<center><h1>Zepto Internship Assignment</h1></center>
<center><h3>Anirvan Krishna | 21EE38002</h3></center>

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from nltk.tokenize import word_tokenize
import nltk
import pickle as pkl

## Data Reading and Analysis

In [2]:
data = pd.read_csv('session_data.csv')
data.head() 

,session_id,product_name
0,000ed966131fcb96e0efc4ff2b716a3e,beetroot
1,000ed966131fcb96e0efc4ff2b716a3e,cucumber
2,0013eab657eaf2d82d7f1e13023d95c2,onion
3,0013eab657eaf2d82d7f1e13023d95c2,long shelf life milk
4,0013fabde1e543dd541be925266aadbc,dates


In [3]:
len(data)

345152

In [4]:
len(data['session_id'].unique())

165335

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345152 entries, 0 to 345151
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   session_id    345152 non-null  object
 1   product_name  345152 non-null  object
dtypes: object(2)
memory usage: 5.3+ MB


In [6]:
# Check for missing values
missing_values = data.isnull().sum()
missing_values

session_id      0
product_name    0
dtype: int64

In [21]:
num_unique_sessions = data['session_id'].nunique()
num_unique_products = data['product_name'].nunique()
products_per_session = data.groupby('session_id').size()

In [22]:
num_unique_sessions

165335

In [24]:
np.min(products_per_session)

2

In [25]:
np.max(products_per_session)

9

In [26]:
num_unique_products

396

In [8]:
data['product_name'].unique()

array(['beetroot', 'cucumber', 'onion', 'long shelf life milk', 'dates',
       'matchbox', 'brinjal', 'gourds', 'notebook', 'rain accessories',
       'earbuds', 'bhujia & sev', 'roasted chana & fried gram',
       'wafer cubes', 'detergent powder', 'garlic', 'fresh cow milk',
       'oats', 'male condoms', 'sooji & rawa', 'chocolates', 'cakes',
       'perfume gift sets', 'almond', 'curd', 'chicken boneless cut',
       'chocolate cone', 'shared pack chocolates', 'tender coconut',
       'baby fruits', 'body soaps', 'watermelon', 'deodorant body spray',
       'cream biscuits', 'chickpea & kabuli chana',
       'buffalo ghee, pure ghee & desi ghee', 'fruit tub',
       'soapcase dispensers & more', 'milk breads', 'coconut oil',
       'laddu', 'hair color cream', 'soya chunks', 'diwali items', 'salt',
       'cat wet food', 'flavoured crackers', 'premium chocolates',
       'gulab jamun pre-mix', 'rice', 'starters', 'soup',
       'instant cup veg noodles', 'face accessories', 'meat 

## Collaborative Filtering

In [ ]:
# Build the interaction matrix for collaborative filtering
interaction_matrix = pd.crosstab(
    data['session_id_encoded'], data['product_name_encoded'])

# Collaborative Filtering with Truncated SVD
n_components = num_unique_products  # Number of latent factors
svd = TruncatedSVD(n_components=n_components)
latent_matrix = svd.fit_transform(interaction_matrix)

# Compute cosine similarity between products
product_similarity = cosine_similarity(latent_matrix.T)

print(f"Number of products: {num_unique_products}")
print(f"Shape of product_similarity matrix: {product_similarity.shape}")
print(f"Index of 'cucumber': {le_product.transform(['cucumber'])[0]}")

# Function to get product recommendation based on Collaborative Filtering
def get_cf_recommendations(product_name, num_recommendations=5):
    product_idx = le_product.transform([product_name])[0]
    if product_idx >= product_similarity.shape[0]:
        raise ValueError(f"Product '{product_name}' not in similarity matrix.")
    product_similarities = product_similarity[product_idx]
    similar_product_indices = product_similarities.argsort()[
        ::-1][:num_recommendations+1]
    similar_products = [le_product.inverse_transform(
        [idx])[0] for idx in similar_product_indices if idx != product_idx][:num_recommendations]
    return similar_products

Number of products: 396
Shape of product_similarity matrix: (396, 396)
Index of 'cucumber': 106


In [66]:
# Example: Get recommendations for a product using Collaborative Filtering
cf_recommendations = get_cf_recommendations("earphones")
print(
    f"Association Rules Recommendations for 'cucumber': {cf_recommendations}")

Association Rules Recommendations for 'cucumber': ['flavoured crackers', 'dressings', 'dog wet food', 'digestives', 'curd']


In [ ]:
data['recommended'] = np.zeros(len(data))

for i in range(len(data)):
    data['recommended'][i] = get_cf_recommendations(
        data['product_name'][i])[:3]

data

## Apriori

In [ ]:
# Download NLTK data
nltk.download('punkt')

# Load the dataset
data = pd.read_csv('session_data.csv')

# Exploratory Data Analysis (EDA)
print(data.head())
print(data.isnull().sum())  # Check for missing values
print(data['product_name'].value_counts())  # Product frequency

# Encode session_id and product_name as integers
le_session = LabelEncoder()
le_product = LabelEncoder()
data['session_id_encoded'] = le_session.fit_transform(data['session_id'])
data['product_name_encoded'] = le_product.fit_transform(data['product_name'])

In [36]:
# Association Rule Mining using Apriori Algorithm
# Convert the interaction matrix to a format suitable for Apriori
basket_sets = interaction_matrix.applymap(lambda x: 1 if x > 0 else 0)
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.85)

# Function to get product recommendations based on association rules
def get_ar_recommendations(product_name, num_recommendations=3):
    product_idx = le_product.transform([product_name])[0]
    recommendations = rules[rules['antecedents'].apply(
        lambda x: product_idx in x)]
    recommendations = recommendations.sort_values(
        'confidence', ascending=False)
    recommended_products = []
    for _, row in recommendations.head(num_recommendations).iterrows():
        consequent_products = row['consequents']
        for product in consequent_products:
            recommended_products.append(
                le_product.inverse_transform([product])[0])
    return recommended_products

C:\Users\Anirvan Krishna\AppData\Roaming\Python\Python310\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [73]:
with open('apriori_model.pkl', 'wb') as file:
    pkl.dump((rules, le_product), file)

In [63]:
# Example: Get recommendations for a product using association rules
ar_recommendations = get_ar_recommendations('cucumber')
print(
    f"Association Rules Recommendations for 'cucumber': {ar_recommendations}")

Association Rules Recommendations for 'cucumber': ['beetroot', 'gourds', 'watermelon']


In [46]:
data['recommended'] = np.zeros(len(data))

for i in range(len(data)):
    data['recommended'][i] = get_ar_recommendations(
        data['product_name'][i])[:3]

data

C:\Users\Anirvan Krishna\AppData\Local\Temp\ipykernel_14660\169239602.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['recommended'][i] = get_ar_recommendations(


,session_id,product_name,session_id_encoded,product_name_encoded,recommended
0,000ed966131fcb96e0efc4ff2b716a3e,beetroot,37,30,"[cucumber, gourds, fresh beans]"
1,000ed966131fcb96e0efc4ff2b716a3e,cucumber,37,106,"[beetroot, gourds, watermelon]"
2,0013eab657eaf2d82d7f1e13023d95c2,onion,52,265,"[long shelf life milk, tender coconut, baby fr..."
3,0013eab657eaf2d82d7f1e13023d95c2,long shelf life milk,52,220,[onion]
4,0013fabde1e543dd541be925266aadbc,dates,53,112,[]
...,...,...,...,...,...
345147,ffb73de5b022d80d23ec72a7ce8226fe,long shelf life milk,165170,220,[onion]
345148,ffb73de5b022d80d23ec72a7ce8226fe,nectar juices,165170,256,[]
345149,ffb73de5b022d80d23ec72a7ce8226fe,onion,165170,265,"[long shelf life milk, tender coconut, baby fr..."
345150,ffcb72279f578ca4cb32c089d8fe8aa4,rice,165216,312,[]


## K-Means Clustering 

In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load data
df = pd.read_csv('session_data.csv')

# Encode product names as numeric vectors using LabelEncoder
label_encoder = LabelEncoder()
df['product_id'] = label_encoder.fit_transform(df['product_name'])

# Create session-product matrix with binary indicators
session_product_matrix = df.assign(value=1).pivot_table(
    index='session_id', columns='product_id', values='value', fill_value=0)

In [10]:
df.head()

,session_id,product_name,product_id
0,000ed966131fcb96e0efc4ff2b716a3e,beetroot,30
1,000ed966131fcb96e0efc4ff2b716a3e,cucumber,106
2,0013eab657eaf2d82d7f1e13023d95c2,onion,265
3,0013eab657eaf2d82d7f1e13023d95c2,long shelf life milk,220
4,0013fabde1e543dd541be925266aadbc,dates,112


In [11]:
len(df['product_name'].unique())

396

In [12]:
session_product_matrix

product_id,0,1,2,3,4,5,6,7,8,9,...,386,387,388,389,390,391,392,393,394,395
session_id,,,,,,,,,,,,,,,,,,,,,
0000bc46bf70d6ebb01c8b58f5e619ab,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000163cb20c349ae5d6b9cd3343a221a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0001978dc214c82cb21a5a3d450ba431,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0001c0bc9fcff00481a997d89de3d2e0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
0002276867f6d276dc39a3261e30b2ca,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffed3a44b22a0ad079f5ecf75c38e75,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ffff24612f86e176e835ca50a98182e1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ffffdfdb1e329a1ffde4f7eef3ff01d8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# Calculate the correlation matrix
session_product_matrix.corr()

product_id,0,1,2,3,4,5,6,7,8,9,...,386,387,388,389,390,391,392,393,394,395
product_id,,,,,,,,,,,,,,,,,,,,,
0,1.000000,-0.000151,-0.000185,-0.001544,0.001853,-0.000511,-0.001745,-0.000443,-0.000823,-0.000286,...,-0.001603,-0.000751,-0.003086,-0.000243,-0.000509,-0.000842,-0.002139,-0.000405,0.004297,-0.000309
1,-0.000151,1.000000,-0.000089,0.007478,-0.000823,-0.000245,-0.000838,-0.000213,-0.000395,-0.000137,...,-0.000770,-0.000361,-0.001482,-0.000117,-0.000244,-0.000405,-0.001027,-0.000194,-0.000539,-0.000148
2,-0.000185,-0.000089,1.000000,-0.000908,-0.001007,-0.000300,-0.001026,-0.000261,-0.000484,-0.000168,...,-0.000943,-0.000442,-0.001816,-0.000143,-0.000299,-0.000496,-0.001258,-0.000238,-0.000660,-0.000181
3,-0.001544,0.007478,-0.000908,1.000000,-0.001083,-0.000073,-0.004966,-0.002173,-0.004037,0.002939,...,-0.007083,-0.003686,-0.013072,-0.001192,-0.002497,-0.004133,-0.007551,0.001085,-0.003277,-0.001514
4,0.001853,-0.000823,-0.001007,-0.001083,1.000000,0.006009,-0.008847,0.000123,0.000985,-0.001557,...,-0.006608,-0.004089,-0.010811,-0.001322,-0.000565,-0.001916,-0.006324,-0.002203,-0.001082,-0.001679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,-0.000842,-0.000405,-0.000496,-0.004133,-0.001916,-0.001368,0.003192,-0.001186,0.000555,-0.000766,...,-0.002867,-0.002011,-0.005996,-0.000650,-0.001363,1.000000,0.006087,-0.001083,0.001053,-0.000826
392,-0.002139,-0.001027,-0.001258,-0.007551,-0.006324,0.003601,-0.004548,0.003107,-0.002296,-0.001945,...,-0.008626,-0.003903,-0.019177,0.002065,-0.003460,0.006087,1.000000,-0.002751,0.004500,-0.002097
393,-0.000405,-0.000194,-0.000238,0.001085,-0.002203,-0.000657,0.000479,-0.000570,-0.001058,-0.000368,...,-0.002061,-0.000966,-0.003970,-0.000312,-0.000655,-0.001083,-0.002751,1.000000,-0.001442,-0.000397


In [57]:
from sklearn.cluster import KMeans

# Apply KMeans clustering
kmeans = KMeans(n_clusters=10)
kmeans.fit(session_product_matrix)


# Function to recommend products based on clustering
def recommend_clustering(model, session_id, session_product_matrix, top_n = 5):
    session_cluster = model.predict(session_product_matrix.loc[[session_id]])
    cluster_sessions = session_product_matrix[model.labels_ ==
                                              session_cluster[0]]
    recommendations = cluster_sessions.mean().sort_values(ascending=False).index
    return recommendations[:5]


# Example usage
print(recommend_clustering(kmeans, session_product_matrix.index[0], session_product_matrix))

C:\Users\Anirvan Krishna\AppData\Roaming\Python\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Int64Index([157, 107, 265, 106, 161], dtype='int64', name='product_id')


In [59]:
recommend_clustering(kmeans,
    session_product_matrix.index[0], session_product_matrix, top_n = 5)

Int64Index([157, 107, 265, 106, 161], dtype='int64', name='product_id')

In [15]:
with open('kmeans_model.pkl', 'wb') as file:
    pkl.dump(kmeans, file)

In [146]:
series = df.iloc[0]
series.loc[series != 0]

session_id      000ed966131fcb96e0efc4ff2b716a3e
product_name                            beetroot
product_id                                    30
Name: 0, dtype: object

In [71]:
def recommend_clustering(model, session_id, session_product_matrix, label_encoder, df, top_n=5):
    session_cluster = model.predict(session_product_matrix.loc[[session_id]])
    cluster_sessions = session_product_matrix[model.labels_ ==
                                              session_cluster[0]]
    cluster_mean = cluster_sessions.mean().sort_values(ascending=False)

    # Get products already in the user's cart
    user_products = set(df[df['session_id'] == session_id]['product_id'])

    # Get top recommendations excluding the user's current products
    recommendations = [
        product for product in cluster_mean.index if product not in user_products][:top_n]

    # Convert product ids back to product names
    recommended_product_names = label_encoder.inverse_transform(
        recommendations)
    
    # Filter the data for the specific session_id
    specific_session_data = data[data['session_id']
                             == '000ed966131fcb96e0efc4ff2b716a3e']

    # Extract the product_name(s)
    product_names = specific_session_data['product_name'].unique()

    print(f"Predictions for {product_names}: {recommended_product_names}")
    #return recommended_product_names

In [72]:
recommend_clustering(kmeans, '000ed966131fcb96e0efc4ff2b716a3e', session_product_matrix,
                    label_encoder, df, top_n=3)

Predictions for ['beetroot' 'cucumber']: ['watermelon' 'garlic' 'gourds']
